In [1]:
""" this notebook provides etl to extract and transform camps and clinics data"""

In [2]:
import os
import pandas

In [ ]:
from data_helper import get_csv_dataframes_from_folder
from data_helper import set_col_names_lower,set_col_names_underscore_separated

In [3]:
csv_dfs = get_csv_dataframes_from_folder('../data/raw/campsandclinincs/airtable')

In [5]:
csv_dfs.keys()

dict_keys(['2024-04-04 12:51:37.csv', 'airtable.csv', '2024-04-04 12:32:32.csv', '2024-04-04 12:58:53.csv'])

In [ ]:
list_of_dfs = [df for file_name, df in csv_dfs.items()]
# for file_name, df in csv_dfs.items():
